In [17]:

%matplotlib inline
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from scipy import stats
warnings.filterwarnings('ignore')
import os
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
import glob
from keras.models import Sequential
from keras.layers import LSTM, Dense, Flatten, Dropout, Conv2D  ,MaxPooling2D
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.optimizers import Adam
import tensorflow as tf

### Preparing Data

In [31]:
respeck_filepaths = glob.glob("../Respeck/*")
df1 = pd.DataFrame()
for rfp in respeck_filepaths:
    files = glob.glob(f"{rfp}/*")
    
    for file in files:
        [main_act,sub_act] = file.split(".csv")[0].split('_')[-2:]
        # main_activity = file.split(".csv")[0].split('_')[-2]
        
        df = pd.read_csv(file,index_col=0)
        df['main_activity'] = main_act
        df['sub_activity'] = sub_act
        df['activity'] = " ".join([main_act,sub_act])
        df['user'] = rfp.split('\\')[-1]
        # print(df)
        df1 = df1.append(df)


In [43]:
classes = ['lyingBack breathingNormal', 'lyingBack coughing',
       'lyingBack hyperventilating', 'lyingLeft breathingNormal',
       'lyingLeft coughing', 'lyingLeft hyperventilating',
       'lyingRight breathingNormal', 'lyingRight coughing',
       'lyingRight hyperventilating', 'lyingStomach breathingNormal',
       'lyingStomach coughing', 'lyingStomach hyperventilating',
       'sitting breathingNormal', 'sitting coughing',
       'sitting hyperventilating', 'standing breathingNormal',
       'standing coughing', 'standing hyperventilating']


df1 = df1[df1['activity'].isin(classes)]     
df1.loc[df1['main_activity'].isin(('sitting', 'standing')),'main_activity'] = 'sitting_standing'

df1['activity'] = df1[['main_activity', 'sub_activity']].agg(' '.join, axis=1) 
df1['activity'].unique()

array(['lyingBack breathingNormal', 'lyingBack coughing',
       'lyingBack hyperventilating', 'lyingLeft breathingNormal',
       'lyingLeft coughing', 'lyingLeft hyperventilating',
       'lyingRight breathingNormal', 'lyingRight coughing',
       'lyingRight hyperventilating', 'lyingStomach breathingNormal',
       'lyingStomach coughing', 'lyingStomach hyperventilating',
       'sitting_standing breathingNormal', 'sitting_standing coughing',
       'sitting_standing hyperventilating'], dtype=object)

In [45]:
columns = ['user','activity','timestamp', 'accel_x', 'accel_y', 'accel_z']

# df1 = df1[columns]
df_har = df1[columns]
# removing null values
df_har = df_har.dropna()
df_har.shape
# transforming the user to float
df_har['user'] = df_har['user'].str.replace('s', '')
df_har['user'] = df_har['user'].apply(lambda x:int(x))
df_har['activity'].unique().shape


(15,)

In [46]:
df_har['activity'].unique()

array(['lyingBack breathingNormal', 'lyingBack coughing',
       'lyingBack hyperventilating', 'lyingLeft breathingNormal',
       'lyingLeft coughing', 'lyingLeft hyperventilating',
       'lyingRight breathingNormal', 'lyingRight coughing',
       'lyingRight hyperventilating', 'lyingStomach breathingNormal',
       'lyingStomach coughing', 'lyingStomach hyperventilating',
       'sitting_standing breathingNormal', 'sitting_standing coughing',
       'sitting_standing hyperventilating'], dtype=object)

In [47]:
df_har.to_csv('task2.csv',index=False)

In [18]:
# ONLY RUN THIS AFTER CSV GENERATION
stationary_respiratory_df = pd.read_csv('task2.csv')
stationary_respiratory_df

,user,activity,timestamp,accel_x,accel_y,accel_z
0,1,lyingBack breathingNormal,1697605965,-0.596436,-0.181213,0.870056
1,1,lyingBack breathingNormal,1697606005,-0.593994,-0.189026,0.880310
2,1,lyingBack breathingNormal,1697606045,-0.591064,-0.172668,0.883484
3,1,lyingBack breathingNormal,1697606085,-0.580322,-0.176819,0.875671
4,1,lyingBack breathingNormal,1697606125,-0.598145,-0.189758,0.882263
...,...,...,...,...,...,...
957994,98,sitting_standing hyperventilating,1697636165,-0.170898,-0.940979,0.119324
957995,98,sitting_standing hyperventilating,1697636205,-0.153809,-0.949524,0.121521
957996,98,sitting_standing hyperventilating,1697636245,-0.188965,-0.937073,0.094666
957997,98,sitting_standing hyperventilating,1697636285,-0.133789,-0.999573,0.073181


### Segment Generation

In [19]:
random_seed = 42   
n_time_steps = 50 
n_features = 3 
step = 10 
n_epochs = 20      
batch_size = 32

In [26]:
def segments_overlap(data):
    segments = []
    labels = []

    for i in range(0,  data.shape[0]- n_time_steps, step):  

        xs = data['accel_x'].values[i: i + n_time_steps]

        ys = data['accel_y'].values[i: i + n_time_steps]

        zs = data['accel_z'].values[i: i + n_time_steps]

        label = stats.mode(data['activity'][i: i + n_time_steps])[0][0]

        segments.append([xs, ys, zs])

        labels.append(label)
        
    reshaped_segments = np.asarray(segments, dtype= np.float32).reshape(-1, n_time_steps, n_features)
    labels = np.asarray(labels).reshape(-1,1)

    enc = OneHotEncoder(handle_unknown='ignore').fit(labels)
    labels = enc.transform(labels).toarray()
    # labels = np.asarray(pd.get_dummies(labels), dtype = np.float32)
    # print(enc.categories_)
    return reshaped_segments,labels,enc.categories_

In [21]:
def segments_no_overlap(data):
    segments= []
    labels = []
    
    activities = data['activity'].unique()
    for act in activities:
        act_data = data[data['activity'] == act]
        
        # for i in range(0, len(all_data) - n_time_steps, step):
        for i in range(0,  act_data.shape[0]- n_time_steps, step):  

            xs = act_data['accel_x'].values[i: i + n_time_steps]
            ys = act_data['accel_y'].values[i: i + n_time_steps]
            zs = act_data['accel_z'].values[i: i + n_time_steps]

            segments.append([xs, ys, zs])
            labels.append(act)

    #reshape the segments which is (list of arrays) to a list
    reshaped_segments = np.asarray(segments, dtype= np.float32).reshape(-1, n_time_steps, n_features)

    labels = np.asarray(pd.get_dummies(labels), dtype = np.float32)
    
    return reshaped_segments,labels

### Model Training

In [22]:
from sklearn.model_selection import train_test_split

def test_train_split(seg,labls):    
    X_train, X_test, y_train, y_test = train_test_split(seg, labls, test_size = 0.2, random_state = random_seed)
    return X_train, X_test, y_train, y_test

In [23]:
def model_LSTM(X_train,y_train):
    model = Sequential()
    # RNN layer
    model.add(LSTM(units = 128, input_shape = (X_train.shape[1], X_train.shape[2])))
    # Dropout layer
    model.add(Dropout(0.5)) 
    # Dense layer with ReLu
    model.add(Dense(units = 64, activation='relu'))
    # Softmax layer
    model.add(Dense(y_train.shape[1], activation = 'softmax'))
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    model.summary()
    return model 

In [24]:
def model_cnn(trainX, trainy):
    n_timesteps, n_features, n_outputs = trainX.shape[1], trainX.shape[2], trainy.shape[1]
    model = Sequential()
    model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(n_timesteps,n_features)))
    model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
    model.add(Dropout(0.5))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(100, activation='relu'))
    model.add(Dense(n_outputs, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    # fit network
    model.fit(trainX, trainy, epochs=n_epochs, batch_size=batch_size, verbose=1)
    # evaluate model
    return model

### Driver Code

In [27]:
accuracies = []
for user in stationary_respiratory_df['user'].unique():
    
    train_df = stationary_respiratory_df[stationary_respiratory_df['user'] != user]
    test_df = stationary_respiratory_df[stationary_respiratory_df['user'] == user]

    X_train, y_train, categories = segments_overlap(train_df)
    X_test, y_test, _ = segments_overlap(test_df)
    
    # model = model_LSTM(X_train,y_train)
    # history = model.fit(X_train, y_train, epochs = n_epochs, validation_split = 0.20, batch_size = batch_size, verbose = 1)
    
    # print(f"Test Accuracy ({user}):", accuracy)
    # print(f"Test Loss ({user}):", loss)
    model  = model_cnn(X_train,y_train)
    
    
    loss, accuracy = model.evaluate(X_test, y_test, batch_size = batch_size, verbose = 1)
    print(f"Test Accuracy ({user}):", accuracy)
    print(f"Test Loss ({user}):", loss)
    
    accuracies.append((user,loss,accuracy))
    break
    

Epoch 1/20
2951/2951 [==============================] - 19s 6ms/step - loss: 0.7012 - accuracy: 0.6946
Epoch 2/20
2951/2951 [==============================] - 18s 6ms/step - loss: 0.4909 - accuracy: 0.7969
Epoch 3/20
2951/2951 [==============================] - 18s 6ms/step - loss: 0.4259 - accuracy: 0.8238
Epoch 4/20
2951/2951 [==============================] - 18s 6ms/step - loss: 0.3836 - accuracy: 0.8423
Epoch 5/20
2951/2951 [==============================] - 20s 7ms/step - loss: 0.3489 - accuracy: 0.8591
Epoch 6/20
2951/2951 [==============================] - 21s 7ms/step - loss: 0.3218 - accuracy: 0.8690
Epoch 7/20
2951/2951 [==============================] - 20s 7ms/step - loss: 0.2989 - accuracy: 0.8788
Epoch 8/20
2951/2951 [==============================] - 20s 7ms/step - loss: 0.2815 - accuracy: 0.8868
Epoch 9/20
2951/2951 [==============================] - 19s 7ms/step - loss: 0.2694 - accuracy: 0.8915
Epoch 10/20
2951/2951 [==============================] - 17s 6ms/step - l

In [63]:
loss, accuracy = model.evaluate(X_test, y_test, batch_size = batch_size, verbose = 1)
accuracy

43/43 [==============================] - 0s 2ms/step - loss: 0.3108 - accuracy: 0.8679


0.8679383993148804

In [29]:
# Convert the model.
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the model.
with open(f'../models/cnn_model_task{2}_{n_time_steps}_{n_features}.tflite', 'wb') as f:
  f.write(tflite_model)

INFO:tensorflow:Assets written to: C:\Users\chhal\AppData\Local\Temp\tmp2azeuqlj\assets


INFO:tensorflow:Assets written to: C:\Users\chhal\AppData\Local\Temp\tmp2azeuqlj\assets


In [28]:
categories

[array(['lyingBack breathingNormal', 'lyingBack coughing',
        'lyingBack hyperventilating', 'lyingLeft breathingNormal',
        'lyingLeft coughing', 'lyingLeft hyperventilating',
        'lyingRight breathingNormal', 'lyingRight coughing',
        'lyingRight hyperventilating', 'lyingStomach breathingNormal',
        'lyingStomach coughing', 'lyingStomach hyperventilating',
        'sitting_standing breathingNormal', 'sitting_standing coughing',
        'sitting_standing hyperventilating'], dtype='<U33')]